In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

%matplotlib inline

In [2]:
dtp_df = pd.read_csv("../Data/SpbData/ProcessingData/dtp_with_boundaries_check.csv")

In [3]:
dtp_df.loc[~dtp_df.in_city_boundaries, 'COORD_L'] = dtp_df[~dtp_df.in_city_boundaries].lng.tolist()
dtp_df.loc[~dtp_df.in_city_boundaries, 'COORD_W'] = dtp_df[~dtp_df.in_city_boundaries].lat.tolist()

In [4]:
geometry = [Point(xy) for xy in zip(dtp_df.COORD_L, dtp_df.COORD_W)]
df = dtp_df.drop(['lat', 'lng'], axis=1)
crs = {'init': 'epsg:4326'}
dtp_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

In [5]:
dtp_gdf = dtp_gdf.dropna(subset=['COORD_L', 'COORD_W'])

In [6]:
spb_boundaries_gdf = gpd.read_file('../Gis/layers/spb_city_boundary.geojson')
spb_boundaries_gdf = spb_boundaries_gdf.drop([1,2])

In [7]:
is_dtp_in_boundaries = dtp_gdf.intersects(spb_boundaries_gdf.unary_union)

In [8]:
dtp_in_city_bound_gdf = dtp_gdf[is_dtp_in_boundaries]

In [14]:
dtp_in_city_bound_gdf = dtp_in_city_bound_gdf.drop('in_city_boundaries',axis=1)

In [15]:
dtp_in_city_bound_gdf.to_file("../Gis/layers/dtp_in_city_bound.geojson", driver='GeoJSON')

In [78]:
#Получим координаты перекрестков
crossroads_gdf = gpd.read_file("../Gis/layers/spb_crossing.geojson")

In [93]:
crossroads_buffer_gdf = gpd.GeoDataFrame(gpd.GeoSeries(crossroads_gdf.buffer(50).envelope))
crossroads_buffer_gdf = crossroads_buffer_gdf.rename(columns={0:'geometry'}).set_geometry('geometry')

In [ ]:
crossroad_union_gdf = gpd.overlay(crossroads_buffer_gdf, crossroads_buffer_gdf, how='union')

In [ ]:
2+2